In [1]:
from keras.applications import MobileNet
img_rows, img_cols = 224, 224 
model = MobileNet(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

Using TensorFlow backend.


In [2]:
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 ZeroPadding2D True
2 Conv2D True
3 BatchNormalization True
4 ReLU True
5 DepthwiseConv2D True
6 BatchNormalization True
7 ReLU True
8 Conv2D True
9 BatchNormalization True
10 ReLU True
11 ZeroPadding2D True
12 DepthwiseConv2D True
13 BatchNormalization True
14 ReLU True
15 Conv2D True
16 BatchNormalization True
17 ReLU True
18 DepthwiseConv2D True
19 BatchNormalization True
20 ReLU True
21 Conv2D True
22 BatchNormalization True
23 ReLU True
24 ZeroPadding2D True
25 DepthwiseConv2D True
26 BatchNormalization True
27 ReLU True
28 Conv2D True
29 BatchNormalization True
30 ReLU True
31 DepthwiseConv2D True
32 BatchNormalization True
33 ReLU True
34 Conv2D True
35 BatchNormalization True
36 ReLU True
37 ZeroPadding2D True
38 DepthwiseConv2D True
39 BatchNormalization True
40 ReLU True
41 Conv2D True
42 BatchNormalization True
43 ReLU True
44 DepthwiseConv2D True
45 BatchNormalization True
46 ReLU True
47 Conv2D True
48 BatchNormalization True
49 ReLU True
50 DepthwiseCo

In [3]:
for layer in model.layers:
    layer.trainable = False

In [4]:
def lw(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

# Set our class number to 3 (Young, Middle, Old)
num_classes = 4

FC_Head = lw(model, num_classes)

model = Model(inputs = model.input, outputs = FC_Head)

print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [6]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'Faces/train/'
validation_data_dir = 'Faces/validation/'

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# set our batch size (typically on most mid tier systems we'll use 16-32)
batch_size = 32
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=1,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=1,
        class_mode='categorical')

Found 1065 images belonging to 4 classes.
Found 412 images belonging to 4 classes.


In [8]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

                     
checkpoint = ModelCheckpoint("faces_mobileNet.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr = 0.001),
              metrics = ['accuracy'])

nb_train_samples = 1097
nb_validation_samples = 272
epochs = 10
batch_size = 16

history = model.fit_generator(
    train_generator,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator)

Epoch 1/10
1065/1065 [==============================] - 299s 281ms/step - loss: 1.4005 - accuracy: 0.2254 - val_loss: 0.4424 - val_accuracy: 0.2136

Epoch 00001: val_loss improved from inf to 0.44244, saving model to faces_mobileNet.h5
Epoch 2/10
1065/1065 [==============================] - 290s 273ms/step - loss: 1.3926 - accuracy: 0.2695 - val_loss: 0.9689 - val_accuracy: 0.3471

Epoch 00002: val_loss did not improve from 0.44244
Epoch 3/10
1065/1065 [==============================] - 287s 270ms/step - loss: 1.3883 - accuracy: 0.2638 - val_loss: 1.6239 - val_accuracy: 0.2646

Epoch 00003: val_loss did not improve from 0.44244
Epoch 4/10
1065/1065 [==============================] - 286s 269ms/step - loss: 1.3884 - accuracy: 0.2657 - val_loss: 0.9084 - val_accuracy: 0.2646
Restoring model weights from the end of the best epoch

Epoch 00004: val_loss did not improve from 0.44244
Epoch 00004: early stopping


In [10]:
train_generator.class_indices

{'Ashish': 0, 'Poorvi': 1, 'Pushpa': 2, 'Shobha': 3}

In [11]:
train_generator.shape

AttributeError: 'DirectoryIterator' object has no attribute 'shape'

In [12]:
len(train_generator)

802